In [ ]:
import json
import sys
from typing import Dict, List, Tuple

def load_snapshots(path: str) -> Dict[int, List[dict]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)  # expects a list of {"_id": ..., "users": [...]}
    by_id = {}
    for item in data:
        _id = item["_id"]
        users = item.get("users", [])
        by_id[_id] = users
    return by_id

def normalize_users(users: List[dict]) -> List[Tuple[str, int, int]]:
    # Convert to sortable, order-insensitive structure
    # Assumes each user has userId (str), score (int), lastUpdateTime (int)
    norm = []
    for u in users:
        norm.append((
            str(u.get("userId")),
            int(u.get("score", 0)),
            int(u.get("lastUpdateTime", 0)),
        ))
    # Sort for deterministic comparison
    norm.sort()
    return norm

def main(file1: str, file2: str):
    snapshots1 = load_snapshots(file1)
    snapshots2 = load_snapshots(file2)

    total_ids = len(snapshots1)
    if total_ids == 0:
        print("File 1 không có _id nào. Tỉ lệ trùng: 0.00%")
        return

    matched_equal = 0
    matched_ids = 0
    missing_in_file2 = []
    different_users = []

    for _id, users1 in snapshots1.items():
        if _id not in snapshots2:
            missing_in_file2.append(_id)
            continue

        matched_ids += 1
        users2 = snapshots2[_id]
        if normalize_users(users1) == normalize_users(users2):
            matched_equal += 1
        else:
            different_users.append(_id)

    percent_equal = matched_equal / total_ids * 100.0
    percent_id_overlap = matched_ids / total_ids * 100.0

    print(f"Tổng _id trong file 1: {total_ids}")
    print(f"_id có mặt trong file 2: {matched_ids} ({percent_id_overlap:.2f}%)")
    print(f"_id mà users giống hệt: {matched_equal}/{total_ids} ({percent_equal:.2f}%)")

    if missing_in_file2:
        print(f"_id có trong file 1 nhưng thiếu ở file 2 (ví dụ 10 đầu): {missing_in_file2[:10]}")
    if different_users:
        print(f"_id có ở cả 2 nhưng users khác nhau (ví dụ 10 đầu): {different_users[:10]}")

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Cách dùng: python compare_snapshots.py <file1.json> <file2.json>")
        sys.exit(1)
    main(sys.argv[1], sys.argv[2])